#### GPT2的手写代码实现

In [1]:
import torch
from torch import nn
from torch import functional as F
from torch.utils.data import Dataset
from torch.utils.data import dataloader
from dataclasses import dataclass

# 设置随机数种子，保证随机数相同
torch.manual_seed(1024)

In [ ]:
@dataclass
class GPTconfig:


In [ ]:
import tiktoken

# 获取 GPT-2 的编码器
enc = tiktoken.get_encoding("gpt2")

# 计算 <|endoftext|> 的 token ID
eos_token = enc.encode("<|endoftext|>", allowed_special={"<|endoftext|>"})[0]

print("End-of-text token ID:", eos_token)  # 输出: 50256


End-of-text token ID: 50256
